<a href="https://colab.research.google.com/github/rnabilahusna/MP_ProductReview/blob/main/web-scrapping/amz_review_theordinary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Reviews Scraping for The Ordinary from Amazon**

In [23]:
# Import packages
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

In [24]:
# Header to set the requests as a browser requests
headers = {
    'authority': 'www.amazon.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'en-US,en;q=0.9,bn;q=0.8',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

In [25]:
# URL of The amazon Review page
reviews_url = 'https://www.amazon.com/Ordinary-Niacinamide-10-Zinc-30ml/product-reviews/B01MDTVZTZ/'

In [26]:
# Define Page Number
len_page = 999

##**Scraping Functions**

In [27]:
# Extra Data as Html object from amazon Review page
def reviewsHtml(url, len_page):

    # Empty List define to store all pages html data
    soups = []

    # Loop for gather all 3000 reviews from 300 pages via range
    for page_no in range(1, len_page + 1):

        # parameter set as page no to the requests body
        params = {
            'ie': 'UTF8',
            'reviewerType': 'all_reviews',
            'filterByStar': 'critical',
            'pageNumber': page_no,
        }

        # Request make for each page
        response = requests.get(url, headers=headers)

        # Save Html object by using BeautifulSoup4 and lxml parser
        soup = BeautifulSoup(response.text, 'lxml')

        # Add single Html page data in master soups list
        soups.append(soup)

    return soups

In [28]:
# Grab Reviews name, description, date, stars, title from HTML
def getReviews(html_data):

    # Create Empty list to Hold all data
    data_dicts = []

    # Select all Reviews BOX html using css selector
    boxes = html_data.select('div[data-hook="review"]')

    # Iterate all Reviews BOX
    for box in boxes:

        # Select Name using css selector and cleaning text using strip()
        # If Value is empty define value with 'N/A' for all.
        try:
            name = box.select_one('[class="a-profile-name"]').text.strip()
        except Exception as e:
            name = 'N/A'

        try:
            stars = box.select_one('[data-hook="review-star-rating"]').text.strip().split(' out')[0]
        except Exception as e:
            stars = 'N/A'

        try:
            title = box.select_one('[data-hook="review-title"]').text.strip()
        except Exception as e:
            title = 'N/A'

        try:
            # Convert date str to dd/mm/yyy format
            datetime_str = box.select_one('[data-hook="review-date"]').text.strip().split(' on ')[-1]
            date = datetime.strptime(datetime_str, '%B %d, %Y').strftime("%d/%m/%Y")
        except Exception as e:
            date = 'N/A'

        try:
            description = box.select_one('[data-hook="review-body"]').text.strip()
        except Exception as e:
            description = 'N/A'

        # create Dictionary with al review data
        data_dict = {
            'Name' : name,
            'Stars' : stars,
            'Title' : title,
            'Date' : date,
            'Description' : description
        }

        # Add Dictionary in master empty List
        data_dicts.append(data_dict)

    return data_dicts

##Process Data

In [29]:
# Grab all HTML
html_datas = reviewsHtml(reviews_url, len_page)

In [30]:
# Empty List to Hold all reviews data
reviews = []

In [31]:
# Iterate all Html page
for html_data in html_datas:

    # Grab review data
    review = getReviews(html_data)

    # add review data in reviews empty list
    reviews += review

In [32]:
# Create a dataframe with reviews Data
df_reviews = pd.DataFrame(reviews)

In [33]:
df_reviews

,Name,Stars,Title,Date,Description
0,darcie olson,4.0,4.0 out of 5 stars\n99% outstanding!,11/07/2023,I’m a 12 year old girl who >used< to suffer fr...
1,Rachel,5.0,5.0 out of 5 stars\nDon't ever want to go with...,17/11/2023,Love love love this! I put it on after my tone...
2,Eliana espinosa,5.0,5.0 out of 5 stars\nEs único,30/11/2023,"Recomiendo este producto,el cambio que tengo e..."
3,Krishnakanth,5.0,5.0 out of 5 stars\nWorks wonderfully,28/11/2023,Best product of Niacinamide. Works wonderfully
4,ripmyshoes,5.0,5.0 out of 5 stars\nThe perfect cheer,22/08/2023,I have been using TO's 10% niacinamide on and ...
...,...,...,...,...,...
9985,Daniela Guerrero,5.0,5.0 out of 5 stars\nReally good,25/11/2023,It has controlled my greasy skin
9986,firstgirl2,5.0,5.0 out of 5 stars\nBetter Than I Hoped For,28/12/2018,I purchased this B3 gel in desperation. I am 7...
9987,Chennelle Thomas,5.0,5.0 out of 5 stars\nMade my skin radiant and c...,24/10/2023,Really worked to clear and brighten my skin.
9988,Taylor Derrett,5.0,5.0 out of 5 stars\nYES YES YOU NEED IT,02/10/2023,My skin tone was so uneven and my pores we HUG...


##**Export Data Scrapped**

In [35]:
# Save data
df_reviews.to_csv('the-ordinary-reviews.csv', index=False)